<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

# デプロイ

## 「モデル」を構成するファイル

学習したネットワークを実際に利用できるように、学習環境から取り出して、アプリケーションに「デプロイ」しましょう。
DIGITS を閉じたところから始めます。

DIGITS によって、デプロイする必要があるファイルがディレクトリに保存されます。
ファイルは、このディレクトリからダウンロードすることも、場所を指定することもできます。
ここでは、学習したときと同じサーバーにモデルを展開するため、DIGITS が生成するフォルダーパスを指定します。

### <a href="/digits">DIGITS を開く</a>

DIGITS のホームページから、「Dogs vs. Cats」と名付けたモデルを選択します。

モデルの作成直後、学習中、および DIGITS の "model" タブでモデルを選択したときのいずれも、DIGITS ではモデルの「ジョブページ」が表示されます。
このページの左上には "Job Directory" があります。

![](images/ModelJobView.PNG)

**ジョブディレクトリ (上でハイライトされた部分) をコピーし、下のコードブロック内の ##FIXME## を置き換えます。
ディレクトリをコピーしたら、セルを実行し (Shift+Enter)、変数 <code>MODEL_JOB_DIR</code> にこれを格納します。**

In [ ]:
MODEL_JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model
!ls $MODEL_JOB_DIR

コピーと貼り付けが成功すると、そのディレクトリ内のすべてのファイルのリストが表示されます。
以降の手順が表示内容に合わない場合は、コピー/貼り付けの手順を確認してください。

今回も、モデルは、アーキテクチャ (訳注: ネットワークの構造) と重みの 2 つのファイルで構成されています。

アーキテクチャは ```deploy.prototxt``` という名前のファイルです。
重みは最新のスナップショットファイル ```snapshot_iter_#.caffemodel``` にあります。
ここでは、スナップショット番号 735 に 5 回のエポックで学習された重みが含まれています。

In [ ]:
ARCHITECTURE = MODEL_JOB_DIR + '/' + 'deploy.prototxt'
WEIGHTS = MODEL_JOB_DIR + '/' + 'snapshot_iter_735.caffemodel'
print ("Filepath to Architecture = " + ARCHITECTURE)
print("Filepath to weights = "+ WEIGHTS)

次に、現在構築しているプログラムが、これらのファイルを確実に読み込み、処理できるようにする必要があります。
この基本的なデプロイでは、フレームワークをインストール (またはインクルード) して、プログラムがモデルファイルを解釈できるようにする必要があります。
フレームワークのインストールを必要としない環境のデプロイ方法については、このコースで後ほど学習します。
並列処理を活用するために GPU を使用する必要もあります。
先述のように、モデルは数十万の演算で構成されており、並列化によって大きく加速することができます。

In [ ]:
import caffe
caffe.set_mode_gpu()

次は、「net」という名前の「Classifier」オブジェクトを作成しましょう。
ワークフローが一般的であるほど、既存のツールでプロジェクトを作成するのが容易になります。
このケースの画像分類は非常に一般的です。
このため、次のコードブロックは、アーキテクチャファイルと重みファイル、およびデータに関する簡単な情報を受け取り、共通処理を容易に実行できるようにしています。

In [ ]:
# Initialize the Caffe model using the model trained in DIGITS
net = caffe.Classifier(ARCHITECTURE, WEIGHTS,  
                       channel_swap =(2, 1, 0), #Color images have three channels, Red, Green, and Blue.
                       raw_scale=255) #Each pixel value is a number between 0 and 255
                       #Each "channel" of our images are 256 x 256 

Classifier クラスには、「predict」という名前のメソッドが含まれています。
このメソッドは、上の定義にしたがって画像を入力として受け取り、その画像が各カテゴリに属する確率を出力します。

## 期待する形式へ入力を変換: 前処理 (Preprocesing)

簡単なものから始めるために、データセットからラベル付き画像を正しく分類してみましょう。
以下のセルを実行することで、画像を読み込んで表示できます。

In [ ]:
import matplotlib.pyplot as plt #matplotlib.pyplot allows us to visualize results
input_image= caffe.io.load_image('/dli/data/dogscats/train/cats/cat.10941.jpg')
plt.imshow(input_image)
plt.show()

これがその画像ですが、ネットワークが想定している「入力」ではありません。

推論用のデータを準備するには、以下の大原則に従います。

<pre>学習の前に行ったことは、必ず推論の前にも行う。</pre>

前のセクションで、DIGITS でモデルをトレーニングしたときに生成されたファイルを見ました。
このセクションでは、DIGITS がデータセットを作成したときに生成されたファイルについて見ていきます。

先ほど学習に使用した **データセット** のジョブディレクトリは、モデルページ「Dogs and Cats」からデータセットを選択するか、DIGITS の "dataset" タブでデータセットを選択すると表示されます。
モデルのディレクトリと同じ場所にありますが、番号が異なります。

![](images/datasetjobdir.PNG)

このデータセットのジョブディレクトリで ##FIXME## を置き換えて下のコードを実行し、DATA_JOB_DIR を正しいファイルパスに設定して、内容を調べます。

In [ ]:
DATA_JOB_DIR = '##FIXME##'  ## Remember to set this to be the job directory for your model
!ls $DATA_JOB_DIR

今回も、ここには (現時点では) 必要以上の情報があります。
データサイエンスやデータの準備について *学べること* は無限にありますが、それらはさまざまなディープラーニングの問題に対処していく中で明確になっていくでしょう。
このケースでは、DIGITS が学習の前に 2 つの処理を実行しました。これを *前処理 (preprocessing)*といいます。

1) DIGITS は、画像のサイズを 256x256 カラー画像に変更しました。

In [ ]:
import cv2
input_image=cv2.resize(input_image, (256, 256), 0,0)
plt.imshow(input_image)
plt.show()

2) DIGITS は、学習に必要な計算を減らすために、各画像から平均画像を減算することで画像を *正規化* しました。

以下のようにして、平均画像を読み込み、テスト画像からこれを減算します。

In [ ]:
mean_image = caffe.io.load_image(DATA_JOB_DIR+'/mean.jpg')
ready_image = input_image-mean_image

データを元のままの状態で受け取り、ネットワークで想定されているデータに変換しました。
次は、ネットワークが作成する出力の内容を見ていきましょう。

## 順伝播: モデルの使用

ここは重要なポイントです。関数を見てみましょう。

<code>prediction = net.predict([grid_square])</code>

他の[関数](https://www.khanacademy.org/computing/computer-programming/programming#functions)と同じように、<code>net.predict</code> は入力 <code>ready_image</code> を渡して、出力 <code>prediction</code> を返します。
他の関数と違うのは、この関数は手順のリストに従わず、行列演算を層ごとに行って、画像を確率のベクトルに変換するという点です。

以下のセルを実行すると、上のラベル付きデータからの予測が表示されます。

In [ ]:
# make prediction
prediction = net.predict([ready_image])
print prediction

興味深いことに、それほど多くの情報は含まれていません。
ネットワークは正規化された 256x256 のカラー画像を受け取り、長さ 2 のベクトルを生成しました。

## 有用な出力の生成: 後処理 (Postprocessing)

ここまで来れば、必要なものは何でも実際に作成できます。
制約があるとすれば、それはプログラミング経験によるものです。
創造力を発揮する前に、基本的なものを作成しましょう。
このコードは、ネットワークが「猫」の確率より「犬」の確率として高い値を出力するかどうかを判定します。
出力する場合は、シミュレーションした犬用のドアに犬が近付いたときにふさわしい画像が表示されます。
出力しない場合は、ドアに猫がいると判断したときにふさわしい画像が表示されます。

In [ ]:
print("Input image:")
plt.imshow(input_image)
plt.show()

print("Output:")
if prediction.argmax()==0:
    print "Sorry cat:( https://media.giphy.com/media/jb8aFEQk3tADS/giphy.gif"
else:
    print "Welcome dog! https://www.flickr.com/photos/aidras/5379402670"

これで、すべてが用意できました。
犬だけを通すドアが観測するであろう画像をテストできます。

In [ ]:
##Create an input our network expects
input_image= caffe.io.load_image('/dli/data/fromnest.PNG')
input_image=cv2.resize(input_image, (256, 256), 0,0)
ready_image = input_image-mean_image
##Treat our network as a function that takes an input and generates an output
prediction = net.predict([ready_image])
print("Input Image:")
plt.imshow(input_image)
plt.show()
print(prediction)
##Create a useful output
print("Output:")
if prediction.argmax()==0:
    print "Sorry cat:( https://media.giphy.com/media/jb8aFEQk3tADS/giphy.gif"
else:
    print "Welcome dog! https://www.flickr.com/photos/aidras/5379402670"

ここで作成したのは、いわば、犬用ドアという課題のためのシミュレーターでした。
カメラからの入力を受け取り、これをネットワークが想定しているデータに変換し、出力を生成、そしてその出力をユーザーにとって有用なものに変換するアプリケーションを作成しました。

正の出力によって犬用ドアのモーターをコントロールするということも、簡単にできるでしょう。
ディープラーニングに関して必要なものが、これで揃いました。
上のコードブロックで試すことができる他の画像を確認するには、下のコマンドを実行して、テスト画像 (学習に使用されなかった画像) の <code>リスト</code> を表示します。
これらの画像の一部では、間違った分類が出力されます。
十分にテストしたらコースを進めて、パフォーマンスを向上する方法を学んでください。

In [ ]:
!ls /dli/data/dogscats/test

## すべてを統合する

このデプロイプロセスを統合し、Jupyter Notebook の外からどう見えるか確認しましょう。
[pythondeployment.py](../../../../edit/tasks/task3/task/pythondeployment.py) の Python ファイルには上と同じコードがありますが、1 つのファイルに統合されています。
コース評価の最後にこの手法を使用するため、見ておいてください。
テスト画像を可視化するには、ファイルパスを以下に追加してください。

In [ ]:
TEST_IMAGE = '/dli/data/dogscats/test/1.jpg'
display= caffe.io.load_image(TEST_IMAGE)
plt.imshow(display)
plt.show()

次に、以下のようにして、その画像を入力とする小さい Python アプリケーションを実行します。
出力の大半は無視して、下までスクロールします (エラーや警告も気にしないでください)。

In [ ]:
!python pythondeployment.py $TEST_IMAGE 2>/dev/null

<a href="https://www.nvidia.com/en-us/deep-learning-ai/education/"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>